In [52]:
from datetime import datetime
import ssl
from config import datalake_url
from pymongo import MongoClient
from pprint import pprint

# Create connection objects

In [53]:
dl_atlas_conn = MongoClient(datalake_url, ssl_cert_reqs=ssl.CERT_NONE)['esteininger-personal-datalake-atlas']
dl_s3_conn = MongoClient(datalake_url, ssl_cert_reqs=ssl.CERT_NONE)['esteininger-personal-datalake-s3']

# Out to S3

In [ ]:
# splitting collection in half
pipeline = [
    {
        '$match': {
            'inc': {
                '$lt': 25
            }
        }
    },
    {
        '$out': {
            's3': {
                'bucket': 'esteininger-personal-datalake',
                'filename': 'analytics',
                "region": "us-east-2",
                'format': {
                    'name': 'json'
                }
            }
        }
    }
]

c = dl_atlas_conn.clickstream.aggregate(pipeline)
print(list(c))
print('Archive created!')


# Create the schema

In [57]:
s3_schema = dl_s3_conn.command({'sqlGenerateSchema': 1, 'setSchemas': True})
# can exclude 
atlas_schema = dl_atlas_conn.command({'sqlGenerateSchema': 1, 'setSchemas': True})

# print("s3 sql schema", s3_schema)
print("atlas sql schema", atlas_schema)

atlas sql schema {'ok': 1, 'schemas': [{'databaseName': 'esteininger-personal-datalake-atlas', 'namespaces': [{'name': 'clickstream', 'schema': {'version': 1, 'jsonSchema': {'bsonType': ['object'], 'properties': {'event': {'bsonType': ['string']}, 'properties': {'bsonType': ['object'], 'properties': {'title': {'bsonType': ['string']}, 'url': {'bsonType': ['string']}, 'path': {'bsonType': ['string']}, 'referrer': {'bsonType': ['string']}, 'foo': {'bsonType': ['string']}, 'lorem': {'bsonType': ['string']}}}, 'user_id': {'bsonType': ['string']}, 'meta': {'bsonType': ['object'], 'properties': {'timestamp': {'bsonType': ['double']}}}, 'inc': {'bsonType': ['int']}, 'options': {'bsonType': ['object']}, '_id': {'bsonType': ['objectId']}}}}}, {'name': 'enriched', 'schema': {'version': 1, 'jsonSchema': {'bsonType': ['object'], 'properties': {'_id': {'bsonType': ['string']}, 'events': {'bsonType': ['array'], 'items': [{'bsonType': ['object'], 'properties': {'properties': {'bsonType': ['object'], 

# Show the Schema was created

In [61]:

# def show_tables(connection_obj):
#     sql = 'select TABLE_NAME from INFORMATION_SCHEMA.TABLES'
#     pipeline = [
#         {
#             '$sql': {
#                 'statement': sql,
#                 'format': "jdbc",
#                 'dialect': "mysql",
#             }
#         }
#     ]
#     print(list(connection_obj.aggregate(pipeline)))
    
c = dl_atlas_conn.command({'sqlGetSchema': "clickstream"})
print(c)
    
# presented as "json schema validators"

# show s3 schema
# show_tables(dl_s3_conn)

# show atlas schema
# show_tables(dl_atlas_conn)

{'ok': 1, 'metadata': {'description': 'set using sqlGenerateSchema with setSchemas = true'}, 'schema': {'version': 1, 'jsonSchema': {'bsonType': ['object'], 'properties': {'event': {'bsonType': ['string']}, 'properties': {'bsonType': ['object'], 'properties': {'title': {'bsonType': ['string']}, 'url': {'bsonType': ['string']}, 'path': {'bsonType': ['string']}, 'referrer': {'bsonType': ['string']}, 'foo': {'bsonType': ['string']}, 'lorem': {'bsonType': ['string']}}}, 'user_id': {'bsonType': ['string']}, 'meta': {'bsonType': ['object'], 'properties': {'timestamp': {'bsonType': ['double']}}}, 'inc': {'bsonType': ['int']}, 'options': {'bsonType': ['object']}, '_id': {'bsonType': ['objectId']}}}}}


# Query MongoDB cluster

In [62]:
sql = "select * from `clickstream`"

pipeline = [
    {
        '$sql': {
            'statement': sql,
            'format': "jdbc",
            'dialect': "mysql",
        }
    }
]

a_sql_q = dl_atlas_conn.aggregate(pipeline)


pprint(list(a_sql_q))

[{'values': [{'bsonType': 'objectId',
              'column': '_id',
              'columnAlias': '_id',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': ObjectId('5f68ba9d7f2fce2fedd6d17d')},
             {'bsonType': 'string',
              'column': 'event',
              'columnAlias': 'event',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'pageStart'},
             {'bsonType': 'long',
              'column': 'inc',
              'columnAlias': 'inc',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 0},
             {'bsonType': 'double',
              'column': 'meta.timestamp',
              'columnAlias': 'meta.timestamp'

              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'ecc4e06d-cdd7-4c73-95a7-2e7171bf6178'}]},
 {'values': [{'bsonType': 'objectId',
              'column': '_id',
              'columnAlias': '_id',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': ObjectId('5f68ba9d7f2fce2fedd6d188')},
             {'bsonType': 'string',
              'column': 'event',
              'columnAlias': 'event',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'buttonClicked'},
             {'bsonType': 'long',
              'column': 'inc',
              'columnAlias': 'inc',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',

             {'bsonType': 'long',
              'column': 'inc',
              'columnAlias': 'inc',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 5},
             {'bsonType': 'double',
              'column': 'meta.timestamp',
              'columnAlias': 'meta.timestamp',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 1600699037.37347},
             {'bsonType': 'string',
              'column': 'properties.foo',
              'columnAlias': 'properties.foo',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.lorem',
              'columnAlias': 'properties.lorem',
            

              'columnAlias': 'properties.foo',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.lorem',
              'columnAlias': 'properties.lorem',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.path',
              'columnAlias': 'properties.path',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': '/'},
             {'bsonType': 'string',
              'column': 'properties.referrer',
              'columnAlias': 'properties.referrer',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clic

              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.referrer',
              'columnAlias': 'properties.referrer',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.title',
              'columnAlias': 'properties.title',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.url',
              'columnAlias': 'properties.url',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'user_id',
              'columnAlias': 'user_id',
              '

...

              'value': 'a73af0d5-eb11-4fbe-8209-b8602e8209a6'}]},
 {'values': [{'bsonType': 'objectId',
              'column': '_id',
              'columnAlias': '_id',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': ObjectId('5f68ba9d7f2fce2fedd6d225')},
             {'bsonType': 'string',
              'column': 'event',
              'columnAlias': 'event',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'pageStart'},
             {'bsonType': 'long',
              'column': 'inc',
              'columnAlias': 'inc',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 42},
             {'bsonType': 'double',
              'colum

              'columnAlias': '_id',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': ObjectId('5f68ba9d7f2fce2fedd6d22f')},
             {'bsonType': 'string',
              'column': 'event',
              'columnAlias': 'event',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'pageStart'},
             {'bsonType': 'long',
              'column': 'inc',
              'columnAlias': 'inc',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 44},
             {'bsonType': 'double',
              'column': 'meta.timestamp',
              'columnAlias': 'meta.timestamp',
              'database': 'esteininger-personal-datalake-atlas',
 

             {'bsonType': 'string',
              'column': 'user_id',
              'columnAlias': 'user_id',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'ebf192ad-c714-4fd4-b219-4855ab8a9b8b'}]},
 {'values': [{'bsonType': 'objectId',
              'column': '_id',
              'columnAlias': '_id',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': ObjectId('5f68ba9d7f2fce2fedd6d236')},
             {'bsonType': 'string',
              'column': 'event',
              'columnAlias': 'event',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'buttonClicked'},
             {'bsonType': 'long',
              'column': 'inc',
     

              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.lorem',
              'columnAlias': 'properties.lorem',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.path',
              'columnAlias': 'properties.path',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': '/'},
             {'bsonType': 'string',
              'column': 'properties.referrer',
              'columnAlias': 'properties.referrer',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream',
              'value': 'https://brokercheck.finra.org/'},
 

             {'bsonType': 'string',
              'column': 'properties.referrer',
              'columnAlias': 'properties.referrer',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.title',
              'columnAlias': 'properties.title',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'properties.url',
              'columnAlias': 'properties.url',
              'database': 'esteininger-personal-datalake-atlas',
              'table': 'clickstream',
              'tableAlias': 'clickstream'},
             {'bsonType': 'string',
              'column': 'user_id',
              'columnAlias': 'user_id',
              'database': 'esteininger-personal-datalake-at

# Query S3 via Atlas Data Lake

In [63]:
sql = "select * from `analytics.1` limit 2"

pipeline = [
    {
        '$sql': {
            'statement': sql,
            'format': "jdbc",
            'dialect': "mysql",
        }
    }
]

r = dl_s3_conn.aggregate(pipeline)
pprint(list(r))

[{'values': [{'bsonType': 'objectId',
              'column': '_id',
              'columnAlias': '_id',
              'database': 'esteininger-personal-datalake-s3',
              'table': 'analytics.1',
              'tableAlias': 'analytics.1',
              'value': ObjectId('5f68ba9d7f2fce2fedd6d17d')},
             {'bsonType': 'string',
              'column': 'event',
              'columnAlias': 'event',
              'database': 'esteininger-personal-datalake-s3',
              'table': 'analytics.1',
              'tableAlias': 'analytics.1',
              'value': 'pageStart'},
             {'bsonType': 'long',
              'column': 'inc',
              'columnAlias': 'inc',
              'database': 'esteininger-personal-datalake-s3',
              'table': 'analytics.1',
              'tableAlias': 'analytics.1',
              'value': 0},
             {'bsonType': 'double',
              'column': 'meta.timestamp',
              'columnAlias': 'meta.timestamp',
       